In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import TimeSeriesSplit
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.metrics import root_mean_squared_error

In [2]:
data = pd.read_csv('full_data.csv')

RandomForestRegressor

In [6]:
tss = TimeSeriesSplit(n_splits = 3)
n = 0
rmse = []
model = RandomForestRegressor()

for train_idxs, val_idxs in tss.split(data[data.date_block_num != 34]):
    train_data, val_data = data.iloc[train_idxs], data.iloc[val_idxs]

    X_train = train_data.drop('item_cnt_month_log', axis = 1)
    y_train = train_data['item_cnt_month_log']
    X_val = val_data.drop('item_cnt_month_log', axis = 1)
    y_val = val_data['item_cnt_month_log']

    model.fit(X_train, y_train)
    y_pred = model.predict(X_val)
    rmse.append(root_mean_squared_error(y_pred, y_val))
    print(f'RMSE for split {n+1}: {rmse[n]}')
print(f'Mean RMSE for all splits: {np.mean(rmse)}')

RMSE for split 1: 0.11199380540343483
RMSE for split 1: 0.11199380540343483
RMSE for split 1: 0.11199380540343483
Mean RMSE for all splits: 0.12008755074043402


XGBRegressor

In [12]:
rmse = []
model = XGBRegressor()

for train_idxs, val_idxs in tss.split(data[data.date_block_num != 34]):
    train_data, val_data = data.iloc[train_idxs], data.iloc[val_idxs]

    X_train = train_data.drop('item_cnt_month_log', axis = 1)
    y_train = train_data['item_cnt_month_log']
    X_val = val_data.drop('item_cnt_month_log', axis = 1)
    y_val = val_data['item_cnt_month_log']

    model.fit(X_train, y_train)
    y_pred = model.predict(X_val)
    rmse.append(root_mean_squared_error(y_pred, y_val))
    print(f'RMSE for split {n+1}: {rmse[n]}')
print(f'Mean RMSE for all splits in XGBRegressor model: {np.mean(rmse)}')

RMSE for split 1: 0.1167564087740027
RMSE for split 1: 0.1167564087740027
RMSE for split 1: 0.1167564087740027
Mean RMSE for all splits in XGBRegressor model: 0.12038998939296497


In [13]:
y_pred = model.predict(X_test)
submission = pd.DataFrame({'ID': len(y_pred), 'item_cnt_month': np.expm1(y_pred)})

SVR

In [ ]:
tss = TimeSeriesSplit(n_splits = 3)
n = 0
rmse = []
model = SVR()

for train_idxs, val_idxs in tss.split(data[data.date_block_num != 34]):
    train_data, val_data = data.iloc[train_idxs], data.iloc[val_idxs]

    X_train = train_data.drop('item_cnt_month_log', axis = 1)
    y_train = train_data['item_cnt_month_log']
    X_val = val_data.drop('item_cnt_month_log', axis = 1)
    y_val = val_data['item_cnt_month_log']

    model.fit(X_train, y_train)
    y_pred = model.predict(X_val)
    rmse.append(root_mean_squared_error(y_pred, y_val))
    print(f'RMSE for split {n+1}: {rmse[n]}')
print(f'Mean RMSE for all splits in SVR model: {np.mean(rmse)}')